In [3]:
import pandas as pd
import numpy as np

from scipy import stats

In [4]:
df = pd.read_csv('/home/yukos/work/hse/applied_stats/week4/botswana.tsb', sep='\t')

df

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4357,2,28,7,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4358,4,24,5,protestant,4.0,1.0,1.0,0,NaN,NaN,0,0.0,1.0,0.0,0.0
4359,1,26,0,spirit,5.0,1.0,0.0,1,22.0,7.0,0,0.0,1.0,0.0,0.0


How many different values does religion feature have?

In [5]:
df.nunique()

ceb         14
age         35
educ        21
religion     4
idlnchld    19
knowmeth     2
usemeth      2
evermarr     2
agefm       35
heduc       21
urban        2
electric     2
radio        2
tv           2
bicycle      2
dtype: int64

Many rows have missing features. How many of them would be left if we exclude all containing NaNs?

In [27]:
df[df.isna().any(axis=1)]

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,NaN
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,NaN
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,NaN
5,1,30,5,spirit,5.0,1.0,0.0,24.0,7.0,1,1.0,0.0,0.0,0.0,NaN
6,3,42,4,other,3.0,1.0,0.0,15.0,11.0,1,1.0,0.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4353,9,49,0,protestant,5.0,0.0,0.0,15.0,0.0,0,0.0,1.0,0.0,0.0,NaN
4354,3,31,2,protestant,4.0,1.0,1.0,18.0,0.0,0,0.0,1.0,0.0,0.0,NaN
4355,4,27,6,protestant,4.0,1.0,1.0,17.0,7.0,0,0.0,0.0,0.0,0.0,NaN
4359,1,26,0,spirit,5.0,1.0,0.0,22.0,7.0,0,0.0,1.0,0.0,0.0,NaN


In [7]:
df.dtypes

ceb           int64
age           int64
educ          int64
religion     object
idlnchld    float64
knowmeth    float64
usemeth     float64
evermarr      int64
agefm       float64
heduc       float64
urban         int64
electric    float64
radio       float64
tv          float64
bicycle     float64
dtype: object

Different features have missing values for different reasons, and they need to be treated differently. For example, agefm has missing values when evermarr=0 – for women who were not married. So NaN for this feature means "Not applicable".

In cases like this, when a feature x1

 is not defined on part of the sample, the recommended way of theating NaNs is the following:

    create a new binary feature x2={1,0,x1='Not applicable',elsewhere;

replace "Not applicable" in x1
 with an arbitrary constant c that x1

     does not have as a value anywhere else

Now, if we build a linear regression on both features – y=β0+β1x1+β2x2
, - on objects, where x1 was measured, the model would look like y=β0+β1x, and on objects, where x1 was "Not applicable", we'll get y=β0+β1c+β2

.

The choice of c
 matters only for the interpretation of β2, but not β1

. Let's use this method to handle missing values in agefm and heduc.

    Create a feature nevermarr that is equal to 1 where agefm has missing values.

    Remove the feature evermarr — with nevermarr they add up to a constant, we can't have such features in linear regression model.

    Replace NaNs in agefm with cagefm=0

.

For objects with nevermarr = 1 replace NaNs in heduc with cheduc1=−1

     (you can't use 0 because it's a valid feature value on some of the other observations).

How many missing values do we have for the feature heduc?

In [8]:
df['nevermarr'] = np.nan
df.loc[df['agefm'].isna(),['nevermarr']] = 1

In [9]:
df = df.drop('evermarr', axis=1)

In [10]:
df.loc[df['agefm'].isna(),['agefm']] = 0

In [11]:
df.loc[(df['nevermarr'] == 1) & df['heduc'].isna(), 'heduc'] = -1

In [12]:
df.isna().sum()

ceb             0
age             0
educ            0
religion        0
idlnchld      120
knowmeth        7
usemeth        71
agefm           0
heduc         123
urban           0
electric        3
radio           2
tv              2
bicycle         3
nevermarr    2079
dtype: int64

Let's get rid of the remaining NaNs.

For features idlnchld, heduc and usemeth do the same: create binary indicators of the misisng value (idlnchld_noans, heduc_noans, usemeth_noans), replace NaNs with impossible constant values (cidlnchld=−1, cheduc2=−2 (value -1 we have already used), cusemeth=−1).

The only missing values we have left are in features knowmeth, electric, radio, tv, and bicycle. They are a few, so just remove the rows that contain them. What is the size of the resulting dataset? Multiply the number of rows by the number of columns (including target variable ceb).

In [28]:
df

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,NaN
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,NaN
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,0,16,7,protestant,2.0,1.0,0.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1.0
4357,2,28,7,protestant,4.0,1.0,1.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1.0
4358,4,24,5,protestant,4.0,1.0,1.0,0.0,-1.0,0,0.0,1.0,0.0,0.0,1.0
4359,1,26,0,spirit,5.0,1.0,0.0,22.0,7.0,0,0.0,1.0,0.0,0.0,NaN
